In [225]:
import pandas as pd
import numpy as np
from sklearn import linear_model as lm
import pickle

We can open the pickle file to get pandas dataframes that contain the data we need for the states we have chosen to examine.

In [226]:
state_list = ['Washington', 'Georgia', 'New Jersey', 'Utah']
with open('combined_data_dfs.pickle', 'rb') as file:
    combined_data_dfs = pickle.load(file)

Name each pandas dataframe for its state

In [227]:
utah=pd.DataFrame(combined_data_dfs[3])
nj=pd.DataFrame(combined_data_dfs[2])
ga=pd.DataFrame(combined_data_dfs[1])
was=pd.DataFrame(combined_data_dfs[0])

Calculate the mean distancing deviation in the google data for each month. This creates a mean for every county in the state.

In [228]:
feb_distance=[0]*(len(utah["County"]))
for x in range(0,20):
        feb_distance[x]=np.mean(utah.iloc[x,2:17])
        
mar_distance=[0]*(len(utah["County"]))
for x in range(0,20):
        mar_distance[x]=np.mean(utah.iloc[x,17:48])
        
apr_distance=[0]*(len(utah["County"]))
for x in range(0,20):
        apr_distance[x]=np.mean(utah.iloc[x,48:59])

Certain counties have employment in industries marked as (D) which means the number is confidential. Larger numbers
are estimated but smaller ones are not. We simply zero out the field to impute the data. We also append our distance averages to the dataframe.

In [229]:
for x in range(0,20):
    if utah.iloc[x,64]=="(D)":
        utah.iloc[x,64]=0

for x in range(0,20):
    for n in range(67,92):
        if utah.iloc[x,n]=="(D)":
            utah.iloc[x,n]=0

utah['f_dist']=feb_distance
utah['m_dist']=mar_distance
utah['a_dist']=apr_distance

We create measures of percentages for nonfarm jobs and industries hit hard by coronavirus (accomodations, entertainment, transportation, retail).

In [230]:
nonfarm_pct=[0]*(len(utah["County"]))
covid_hit=[0]*(len(utah["County"]))
for x in range(0,20):
        nonfarm_pct[x]=float(utah.iloc[x,65])/float(utah.iloc[x,59])
        covid_hit[x]=(float(utah.iloc[x,83])+float(utah.iloc[x,84])+float(utah.iloc[x,68])+float(utah.iloc[x,73])+float(utah.iloc[x,74]))/float(utah.iloc[x,59])
utah['nfarm']=nonfarm_pct
utah['hosp']=covid_hit

Exo_list is the contains the index of the independent variable that are used in the regression.

In [231]:
exo_list=list()

exo_list.append(1)#Income
exo_list.append(95)#Nonfarm employment
exo_list.append(96)#Covid affected employment


In order to use Scikit's linear regression model, we need to convert the dataframe to a numpy array and change the types
to floats from the original strings.

In [232]:
Y=utah['a_dist'].to_numpy()
X=utah.iloc[:,exo_list].to_numpy()
for i in range(0, len(Y)): 
    Y[i] = float(Y[i])
for i in range(0,len(X[:,1])):
    for n in range(0,len(X[1,:])):
        X[i,n]=float(X[i,n])

Our regression model is as follows:

percent change from baseline work trips = b_0 + b_1 county income (dollars) + b_2 percent nonfarm employment in county + b_3 percent employment in industries with heavy covid impact.

The ouput from the regression includes the R-squared figure and the coefficients of regression. It also prints the average of the other two features for use in the SEIR model.

In [250]:
utah_reg = lm.LinearRegression().fit(X, Y)
print("R-squared: "+ str(utah_reg.score(X,Y)))
print("Regression Coefficients: " + str(utah_reg.coef_))
print("Regression Intercept: " + str(utah_reg.intercept_))
print("Average Nonfarm Employment Percent: " + str(np.mean(X[:,1])))
print("Average Covid Affected Employment Percent: " + str(np.mean(X[:,2])))
x=round((utah_reg.intercept_ + utah_reg.coef_[1]*np.mean(X[:,1]) + utah_reg.coef_[2]*np.mean(X[:,2])),3)
print("Equation: " + str(x) + str(round(utah_reg.coef_[0],6)) + "*income" )

R-squared: 0.712052348811743
Regression Coefficients: [-3.24337313e-04 -1.16951612e+02  1.93306095e+01]
Regression Intercept: 80.28119700313414
Average Nonfarm Employment Percent: 0.9557809844719977
Average Covid Affected Employment Percent: 0.2659150890506241
Equation: -26.359-0.000324*income


Repeat Process for New Jersey

In [251]:
feb_distance=[0]*(len(nj["County"]))
for x in range(0,19):
        feb_distance[x]=np.mean(nj.iloc[x,2:17])
        
mar_distance=[0]*(len(nj["County"]))
for x in range(0,19):
        mar_distance[x]=np.mean(nj.iloc[x,17:48])
        
apr_distance=[0]*(len(nj["County"]))
for x in range(0,19):
        apr_distance[x]=np.mean(nj.iloc[x,48:59])

for x in range(0,19):
    if nj.iloc[x,64]=="(D)":
        nj.iloc[x,64]=0

for x in range(0,19):
    for n in range(67,92):
        if nj.iloc[x,n]=="(D)":
            nj.iloc[x,n]=0

nj['f_dist']=feb_distance
nj['m_dist']=mar_distance
nj['a_dist']=apr_distance

nonfarm_pct=[0]*(len(nj["County"]))
covid_hit=[0]*(len(nj["County"]))
for x in range(0,19):
        nonfarm_pct[x]=float(nj.iloc[x,65])/float(nj.iloc[x,59])
        covid_hit[x]=(float(nj.iloc[x,83])+float(nj.iloc[x,84])+float(nj.iloc[x,68])+float(nj.iloc[x,73])+float(nj.iloc[x,74]))/float(nj.iloc[x,59])
nj['nfarm']=nonfarm_pct
nj['hosp']=covid_hit

In [252]:
Y=nj['a_dist'].to_numpy()
X=nj.iloc[:,exo_list].to_numpy()
for i in range(0, len(Y)): 
    Y[i] = float(Y[i])
for i in range(0,len(X[:,1])):
    for n in range(0,len(X[1,:])):
        X[i,n]=float(X[i,n])

In [253]:
nj_reg = lm.LinearRegression().fit(X, Y)
print("R-squared: "+ str(nj_reg.score(X,Y)))
print("Regression Coefficients: " + str(nj_reg.coef_))
print("Regression Intercept: " + str(nj_reg.intercept_))
print("Average Nonfarm Employment Percent: " + str(np.mean(X[:,1])))
print("Average Covid Affected Employment Percent: " + str(np.mean(X[:,2])))
x=round((nj_reg.intercept_ + nj_reg.coef_[1]*np.mean(X[:,1]) + nj_reg.coef_[2]*np.mean(X[:,2])),3)
print("Equation: " + str(x) + str(round(nj_reg.coef_[0],6)) + "*income" )

R-squared: 0.7609575230333342
Regression Coefficients: [-2.17762575e-04 -3.14017457e+02 -2.88669379e+01]
Regression Intercept: 277.6678558853536
Average Nonfarm Employment Percent: 0.9909522466151492
Average Covid Affected Employment Percent: 0.23566705971955157
Equation: -40.311-0.000218*income


Repeat Process for Georgia

In [254]:
feb_distance=[0]*(len(ga["County"]))
for x in range(0,138):
        feb_distance[x]=np.mean(ga.iloc[x,2:17])
        
mar_distance=[0]*(len(ga["County"]))
for x in range(0,138):
        mar_distance[x]=np.mean(ga.iloc[x,17:48])
        
apr_distance=[0]*(len(ga["County"]))
for x in range(0,138):
        apr_distance[x]=np.mean(ga.iloc[x,48:59])

for x in range(0,138):
    if ga.iloc[x,64]=="(D)":
        ga.iloc[x,64]=0

for x in range(0,138):
    for n in range(67,92):
        if ga.iloc[x,n]=="(D)":
            ga.iloc[x,n]=0

ga['f_dist']=feb_distance
ga['m_dist']=mar_distance
ga['a_dist']=apr_distance

nonfarm_pct=[0]*(len(ga["County"]))
covid_hit=[0]*(len(ga["County"]))
for x in range(0,138):
        nonfarm_pct[x]=float(ga.iloc[x,65])/float(ga.iloc[x,59])
        covid_hit[x]=(float(ga.iloc[x,83])+float(ga.iloc[x,84])+float(ga.iloc[x,68])+float(ga.iloc[x,73])+float(ga.iloc[x,74]))/float(ga.iloc[x,59])
ga['nfarm']=nonfarm_pct
ga['hosp']=covid_hit

In [255]:
Y=ga['a_dist'].to_numpy()
X=ga.iloc[:,exo_list].to_numpy()
for i in range(0, len(Y)): 
    Y[i] = float(Y[i])
for i in range(0,len(X[:,1])):
    for n in range(0,len(X[1,:])):
        X[i,n]=float(X[i,n])

In [256]:
ga_reg = lm.LinearRegression().fit(X, Y)
print("R-squared: "+ str(ga_reg.score(X,Y)))
print("Regression Coefficients: " + str(ga_reg.coef_))
print("Regression Intercept: " + str(ga_reg.intercept_))
print("Average Nonfarm Employment Percent: " + str(np.mean(X[:,1])))
print("Average Covid Affected Employment Percent: " + str(np.mean(X[:,2])))
x=round((ga_reg.intercept_ + ga_reg.coef_[1]*np.mean(X[:,1]) + ga_reg.coef_[2]*np.mean(X[:,2])),3)
print("Equation: " + str(x) + str(round(ga_reg.coef_[0],6)) + "*income" )

R-squared: 0.44789574626055306
Regression Coefficients: [-3.40252555e-04 -5.89051461e+01 -2.33211159e+00]
Regression Intercept: 31.467840562628552
Average Nonfarm Employment Percent: 0.9618369579207076
Average Covid Affected Employment Percent: 0.1959227171172739
Equation: -25.646-0.00034*income


Repeat Process for Washington

In [257]:
feb_distance=[0]*(len(was["County"]))
for x in range(0,29):
        feb_distance[x]=np.mean(was.iloc[x,2:17])
        
mar_distance=[0]*(len(was["County"]))
for x in range(0,29):
        mar_distance[x]=np.mean(was.iloc[x,17:48])
        
apr_distance=[0]*(len(was["County"]))
for x in range(0,29):
        apr_distance[x]=np.mean(was.iloc[x,48:59])

for x in range(0,29):
    if was.iloc[x,64]=="(D)":
        was.iloc[x,64]=0

for x in range(0,29):
    for n in range(67,92):
        if was.iloc[x,n]=="(D)":
            was.iloc[x,n]=0

was['f_dist']=feb_distance
was['m_dist']=mar_distance
was['a_dist']=apr_distance

nonfarm_pct=[0]*(len(was["County"]))
covid_hit=[0]*(len(was["County"]))
for x in range(0,29):
        nonfarm_pct[x]=float(was.iloc[x,65])/float(was.iloc[x,59])
        covid_hit[x]=(float(was.iloc[x,83])+float(was.iloc[x,84])+float(was.iloc[x,68])+float(was.iloc[x,73])+float(was.iloc[x,74]))/float(was.iloc[x,59])
was['nfarm']=nonfarm_pct
was['hosp']=covid_hit

In [258]:
Y=was['a_dist'].to_numpy()
X=was.iloc[:,exo_list].to_numpy()
for i in range(0, len(Y)): 
    Y[i] = float(Y[i])
for i in range(0,len(X[:,1])):
    for n in range(0,len(X[1,:])):
        X[i,n]=float(X[i,n])

In [259]:
was_reg = lm.LinearRegression().fit(X, Y)
print("R-squared: "+ str(was_reg.score(X,Y)))
print("Regression Coefficients: " + str(was_reg.coef_))
print("Regression Intercept: " + str(was_reg.intercept_))
print("Average Nonfarm Employment Percent: " + str(np.mean(X[:,1])))
print("Average Covid Affected Employment Percent: " + str(np.mean(X[:,2])))
x=round((was_reg.intercept_ + was_reg.coef_[1]*np.mean(X[:,1]) + was_reg.coef_[2]*np.mean(X[:,2])),3)
print("Equation: " + str(x) + str(round(was_reg.coef_[0],6)) + "*income" )

R-squared: 0.5046112183930104
Regression Coefficients: [-3.67987552e-04 -2.62518864e+01 -8.52556358e+00]
Regression Intercept: -1.3139378032408189
Average Nonfarm Employment Percent: 0.9493031221593514
Average Covid Affected Employment Percent: 0.22754792675886057
Equation: -28.175-0.000368*income
